In [ ]:
import numpy as np
import pandas as pd

In [ ]:
project_path = '/content/drive/My Drive/HAX/Fashion'

In [ ]:
test_zip_path = project_path + '/test_ScVgIM0.zip'
train_zip_path = project_path + '/train_LbELtWX.zip'

In [ ]:
from zipfile import ZipFile

with ZipFile(train_zip_path, 'r') as z:
  z.extractall()
print("Training zip extraction done!")

with ZipFile(test_zip_path, 'r') as z:
  z.extractall()
print("Validation zip extraction done!")

Training zip extraction done!
Validation zip extraction done!


In [ ]:
!ls

drive	     sub11.csv	sub3.csv  sub6.csv  sub9.csv	 test.csv
sample_data  sub1.csv	sub4.csv  sub7.csv  sub_sup.csv  train
sub10.csv    sub2.csv	sub5.csv  sub8.csv  test	 train.csv


In [ ]:
import pandas as pd
labels = pd.read_csv('./train.csv')
labels.head()

,id,label
0,1,9
1,2,0
2,3,0
3,4,3
4,5,0


In [ ]:
import numpy as np
targets = pd.Series(labels['label'])
one_hot = pd.get_dummies(targets)
one_hot_labels = np.asarray(one_hot)

In [ ]:
img_rows=28
img_cols=28
num_channel=1
from tqdm import tqdm
import cv2
x_feature = []
y_feature = []

i = 0 # initialisation
for f, img in tqdm(labels.values): # f for format ,jpg
    train_img = cv2.imread('./train/{}.png'.format(f),0)
    label = one_hot_labels[i]
    train_img_resize = cv2.resize(train_img, (img_rows, img_cols)) 
    x_feature.append(train_img_resize)
    y_feature.append(label)
    i += 1

100%|██████████| 60000/60000 [00:03<00:00, 15120.37it/s]


In [ ]:
x_train_data = np.array(x_feature, np.float32) / 255.   # /= 255 for normolisation
print (x_train_data.shape)
x_train_data = np.expand_dims(x_train_data, axis = 3) # for keras to given input to Conv2D layer
print (x_train_data.shape)
y_train_data = np.array(y_feature)
print(y_train_data.shape)

(60000, 28, 28)
(60000, 28, 28, 1)
(60000, 10)


In [ ]:
submission = pd.read_csv('test.csv')
test_img = submission['id']
submission.head()

,id
0,60001
1,60002
2,60003
3,60004
4,60005


In [ ]:
x_test_feature = []

i = 0 # initialisation
for f in tqdm(test_img.values): # f for format ,jpg
    img = cv2.imread('./test/{}.png'.format(f), 0)
    img_resize = cv2.resize(img, (img_rows, img_cols)) 
    x_test_feature.append(img_resize)

100%|██████████| 10000/10000 [00:00<00:00, 15991.27it/s]


In [ ]:
x_test_data = np.array(x_test_feature, np.float32) / 255. 
print (x_test_data.shape)
x_test_data = np.expand_dims(x_test_data, axis = 3)
print (x_test_data.shape)

(10000, 28, 28)
(10000, 28, 28, 1)


In [ ]:
import tensorflow as tf

In [ ]:
def squeeze_excite_block(filters,input):                      
    se = tf.keras.layers.GlobalAveragePooling2D()(input)
    se = tf.keras.layers.Reshape((1, filters))(se) 
    se = tf.keras.layers.Dense(filters//16, activation='relu')(se)
    se = tf.keras.layers.Dense(filters, activation='sigmoid')(se)
    se = tf.keras.layers.multiply([input, se])
    return se

In [ ]:
def make_model():
        s = tf.keras.Input(shape=x_train_data.shape[1:]) 
        x = tf.keras.layers.Conv2D(32,(3,3),activation='relu',padding='same')(s)
        x = tf.keras.layers.Conv2D(32,(3,3),activation='relu',padding='same')(x)
        x = tf.keras.layers.Conv2D(32,(3,3),activation='relu',padding='same')(x)
        x = tf.keras.layers.BatchNormalization()(x)
        x = squeeze_excite_block(32,x)

        x = tf.keras.layers.Conv2D(32,(3,3),activation='relu',padding='same')(x)
        x = tf.keras.layers.Conv2D(32,(3,3),activation='relu',padding='same')(x)
        x = tf.keras.layers.Conv2D(32,(3,3),activation='relu',padding='same')(x)
        x = tf.keras.layers.BatchNormalization()(x)
        x = squeeze_excite_block(32,x)
        x = tf.keras.layers.AveragePooling2D(2)(x)

        x = tf.keras.layers.Conv2D(32,(3,3),activation='relu',padding='same')(x)
        x = tf.keras.layers.Conv2D(32,(3,3),activation='relu',padding='same')(x)
        x = tf.keras.layers.Conv2D(32,(3,3),activation='relu',padding='same')(x)
        x = tf.keras.layers.BatchNormalization()(x)
        x = squeeze_excite_block(32,x)
        x = tf.keras.layers.AveragePooling2D(2)(x)

        x = tf.keras.layers.Conv2D(32,(3,3),activation='relu',padding='same')(x)
        x = tf.keras.layers.Conv2D(32,(3,3),activation='relu',padding='same')(x)
        x = tf.keras.layers.Conv2D(32,(3,3),activation='relu',padding='same')(x)
        x = tf.keras.layers.BatchNormalization()(x)
        x = squeeze_excite_block(32,x)
        x = tf.keras.layers.AveragePooling2D(2)(x)          


        x = tf.keras.layers.Conv2D(32,(3,3),activation='relu',padding='same')(x)
        x = tf.keras.layers.Conv2D(32,(3,3),activation='relu',padding='same')(x)
        x = tf.keras.layers.Conv2D(32,(3,3),activation='relu',padding='same')(x)
        x = tf.keras.layers.BatchNormalization()(x)
        x = squeeze_excite_block(32,x)
        x = tf.keras.layers.AveragePooling2D(2)(x)


        x = tf.keras.layers.concatenate([tf.keras.layers.GlobalMaxPooling2D()(x),
                                         tf.keras.layers.GlobalAveragePooling2D()(x)])

        x = tf.keras.layers.Dense(10,activation='softmax',use_bias=False,
                                  kernel_regularizer=tf.keras.regularizers.l1(0.00025))(x)
        return tf.keras.Model(inputs=s, outputs=x)

In [ ]:
datagen = tf.keras.preprocessing.image.ImageDataGenerator(rotation_range=10, width_shift_range=0.1, shear_range=10,
                        height_shift_range=0.1, zoom_range=0.2)
datagen.fit(x_train_data)

In [ ]:
from tensorflow.keras import optimizers
batch_size=32
supermodel=[]
for i in range(5):
        np.random.seed(i)
        model=make_model()                
        model.compile(optimizer=optimizers.Adam(lr=0.001), loss='categorical_crossentropy',metrics=['accuracy'])
        model.fit_generator(datagen.flow(x_train_data, y_train_data, batch_size=batch_size,shuffle=True),
                    steps_per_epoch=len(x_train_data) / batch_size, epochs=15,verbose=0)
        model.compile(optimizer=optimizers.Adam(lr=0.0001), loss='categorical_crossentropy',metrics=['accuracy'])
        model.fit_generator(datagen.flow(x_train_data, y_train_data, batch_size=batch_size,shuffle=True),
                    steps_per_epoch=len(x_train_data) / batch_size, epochs=3,verbose=0)
        model.compile(optimizer=optimizers.Adam(lr=0.00001), loss='categorical_crossentropy',metrics=['accuracy'])
        model.fit_generator(datagen.flow(x_train_data, y_train_data, batch_size=batch_size,shuffle=True),
                    steps_per_epoch=len(x_train_data) / batch_size, epochs=3,verbose=0)
        model.fit(x_train_data, y_train_data, batch_size=batch_size,shuffle=True, epochs=1,verbose=0)
        supermodel.append(model)

Instructions for updating:
Please use Model.fit, which supports generators.


In [ ]:
pred =np.asarray([a.predict(x_test_data) for a in supermodel])
predictions = np.argmax(np.mean(pred,axis=0),axis=1)

In [ ]:
predict = pd.DataFrame(data=predictions ,columns=["label"])

In [ ]:
submission = pd.read_csv('/content/drive/My Drive/HAX/Fashion/sample_submission_I5njJSF.csv')
sub = submission['id']
DT = pd.merge(sub , predict, on=None, left_index= True,
    right_index=True)

In [ ]:
DT.to_csv('sub_sup1.csv',index = False)

In [ ]:
!cp sub_sup1.csv "drive/My Drive/"